<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/Parameterize_FlattenUnknowns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
import jax.numpy as jnp
import jax

In [2]:
def parameterize(f, **kw):

    params={}
    vars=[]
    for k,v in kw.items():
        if v is None:
            vars.append(k)
        else:
            params[k]=v

    def f_parameterized(v):
        var_dict = {k:v for k,v in zip(vars,v)}
        params.update(var_dict)
        return eqs(**params)


    return f_parameterized

In [3]:
def concat(list_of_vec):
    return np.concatenate([np.ravel(vec) for vec in list_of_vec])

In [4]:
class Unk():

    def __init__(self,x):
        if type(x) is np.ndarray:
            self.shape=x.shape
            self.size = x.size
        else:
            self.shape=tuple()
            self.size=1

In [5]:
def parameterize2(f, **kw):

    params={}
    vars=[]
    shapes=[]
    sizes=[]
    for k,v in kw.items():
        if isinstance(v,Unk):
            vars.append(k)
            shapes.append(v.shape)
            sizes.append(v.size)
        else:
            params[k]=v
    indices=np.cumsum(sizes)

    def create_var_dict(v):
        return {k:v.reshape(shape) for k,shape,v in zip(vars,shapes,np.split(v,indices))}

    def f_parameterized(v):
        var_dict = create_var_dict(v)
        params.update(var_dict)
        return eqs(**params)


    return f_parameterized, create_var_dict, params

In [36]:
def eqs(vtot, mtot, x1, x2, P):
    return (x1+3*x2)+2*P

x1guess = np.array([0.1,0.9])
x2guess = np.array([0.3,0.7])
Pguess = 1e5

v=concat([x1guess, x2guess, Pguess])
f, create_var_dict, params = parameterize2(eqs, x1=Unk(x1guess), x2=Unk(x2guess), P=Unk(Pguess), vtot=10., mtot=2.)

In [37]:
create_var_dict(v)

{'P': array(100000.), 'x1': array([0.1, 0.9]), 'x2': array([0.3, 0.7])}

In [38]:
jax_f=jax.jit(f)

In [39]:
jax_f(v)

DeviceArray([200001., 200003.], dtype=float32)

In [40]:
jax.jacobian(f)(v)

DeviceArray([[1., 0., 3., 0., 2.],
             [0., 1., 0., 3., 2.]], dtype=float32)